# Bayes Estimator

### <font color='orange'>Universidad Autónoma de Yucatán</font> _Facultad de Matemáticas_

**Teacher:** Dr. Arturo Espinosa Romero <[eromero@correo.uady.mx](mailto:eromero@correo.uady.mx)>

**Student:** Ing. Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Practice 5:  Kalman Filter for Ball Tracking

## GitHub: [Practice 5](https://github.com/dayan3847/bayes_estimator/tree/main/practice5-ball_tracking)

In [31]:
import sympy as sp
import numpy as np

In [22]:
# sp.init_printing(use_unicode=True)

In [32]:
#@title Variables:
X, Y, Z = sp.symbols("X Y Z")  # Coordenadas en metros
R = sp.Symbol('R')  # Radio en metros
Xp = sp.Symbol(r'\dot{X}')
Yp = sp.Symbol(r'\dot{Y}')
Zp = sp.Symbol(r'\dot{Z}')
dt = sp.Symbol(r'\Delta t')

In [33]:
#@title Estado de sistema
# El estado esta compuesto por las coordenadas reales y las velocidades
XX = sp.Matrix([
    [X],
    [Y],
    [Z],
    [Xp],
    [Yp],
    [Zp],
])

sp.Eq(sp.Symbol(r'\mathbb{X}'), XX, evaluate=False)

Eq(\mathbb{X}, Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [34]:
#@title Matriz A "transitionMatrix"
# Esta seria la matrix de transformacion de estado
AA = sp.Matrix([
    [1, 0, 0, dt, 0, 0],
    [0, 1, 0, 0, dt, 0],
    [0, 0, 1, 0, 0, dt],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
])
sp.Eq(sp.Symbol(r'\mathbb{A}'), AA, evaluate=False)

Eq(\mathbb{A}, Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]]))

In [38]:
#@title Ecuacion de tranformacion de estado
# el estado en k, es el resultado de multiplicar A por el estado anterior
sp.Eq(XX, sp.MatMul(AA, XX, evaluate=False), evaluate=False)

Eq(Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]), Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]])*Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [40]:
#@title "Z" Medicion
# La medicion esta compuesta por las coordenadas y el tamanno del radio (en metros)
ZZ = sp.Matrix([
    [X],
    [Y],
    [Xp],
    [Yp],
    [R],
])
sp.Eq(sp.Symbol(r'\mathbb{Z}'), ZZ, evaluate=False)

Eq(\mathbb{Z}, Matrix([
[      X],
[      Y],
[\dot{X}],
[\dot{Y}],
[      R]]))

In [46]:
# @title Funcion h
h_xz = sp.Matrix([
    [X / Z],
    [Y / Z],
    [(Xp + (X / Z) * Zp) / Z],
    [(Z * Yp + Y * Zp) / Z ** 2],
    [R / Z],
])
sp.Eq(sp.Function('h')(X, Z), h_xz, evaluate=False)

Eq(h(X, Z), Matrix([
[                         X/Z],
[                         Y/Z],
[   (X*\dot{Z}/Z + \dot{X})/Z],
[(Y*\dot{Z} + Z*\dot{Y})/Z**2],
[                         R/Z]]))

In [47]:
# @title Matriz H es el Jacobian de h respecto al estado
HH = h_xz.jacobian(XX)
sp.Eq(sp.Symbol(r'\mathbb{H}x'), HH, evaluate=False)

Eq(\mathbb{H}x, Matrix([
[         1/Z,            0,                                        -X/Z**2,   0,   0,      0],
[           0,          1/Z,                                        -Y/Z**2,   0,   0,      0],
[\dot{Z}/Z**2,            0, -X*\dot{Z}/Z**3 - (X*\dot{Z}/Z + \dot{X})/Z**2, 1/Z,   0, X/Z**2],
[           0, \dot{Z}/Z**2,  \dot{Y}/Z**2 - 2*(Y*\dot{Z} + Z*\dot{Y})/Z**3,   0, 1/Z, Y/Z**2],
[           0,            0,                                        -R/Z**2,   0,   0,      0]]))

In [48]:
#@title Matriz de calibracion.
K_cam = sp.Matrix([
    [1.3778036814997304e+03, 0, 4.0002681782947193e+02],
    [0, 1.3778036814997304e+03, 3.00096061319675721e+02],
    [0, 0, 1],
])
sp.Eq(sp.Symbol('K'), K_cam, evaluate=False)

Eq(K, Matrix([
[1377.80368149973,                0, 400.026817829472],
[               0, 1377.80368149973, 300.096061319676],
[               0,                0,                1]]))

In [59]:
#@title Matriz de calibracion (inversa)
K_cam_inv = K_cam.inv()
sp.Eq(sp.Symbol('K^{-1}'), K_cam_inv, evaluate=False)

Eq(K^{-1}, Matrix([
[0.000725792805918116,                    0, -0.290336586554948],
[                   0, 0.000725792805918116, -0.217807562390183],
[                   0,                    0,                  1]]))

In [60]:
# imprimir la innversa de K en formato de numpy
print(np.array(K_cam_inv).astype(np.float64))

[[ 7.25792806e-04  0.00000000e+00 -2.90336587e-01]
 [ 0.00000000e+00  7.25792806e-04 -2.17807562e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [56]:
OO  = K_cam_inv * sp.Matrix([
    [399],
    [299],
    [1],
])
OO

Matrix([
[-0.000745256993619159],
[ -0.00079551342066575],
[                    1]])

In [57]:
K_cam * OO

Matrix([
[399.0],
[299.0],
[    1]])

In [ ]:
f = K_cam[0,0]
(Z * Rp) /f